### Load Data

In [35]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Neuron Layer Definition
- created a helper function for generating neuron layers
- this is not necessary, we could just use tf.layers.dense instead and get same results

In [36]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else :
            return Z

### Define Computation Graph

In [37]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [38]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [39]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation= tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [40]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [41]:
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [42]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

### Execute Session

In [46]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [48]:
n_epochs = 50
batch_size = 50

In [49]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print("Epoch:\t", epoch, "Training accuracy:\t", acc_train, "Test accuracy:\t", acc_test)
    save_path = saver.save(sess, "./my_model_final.ckpt")
    print("Training complete, checkpoint saved")

Epoch:	 0 Training accuracy:	 0.9 Test accuracy:	 0.9142
Epoch:	 1 Training accuracy:	 0.94 Test accuracy:	 0.9322
Epoch:	 2 Training accuracy:	 0.94 Test accuracy:	 0.9427
Epoch:	 3 Training accuracy:	 0.94 Test accuracy:	 0.9469
Epoch:	 4 Training accuracy:	 0.96 Test accuracy:	 0.9508
Epoch:	 5 Training accuracy:	 0.98 Test accuracy:	 0.956
Epoch:	 6 Training accuracy:	 0.96 Test accuracy:	 0.9574
Epoch:	 7 Training accuracy:	 0.92 Test accuracy:	 0.9607
Epoch:	 8 Training accuracy:	 1.0 Test accuracy:	 0.9632
Epoch:	 9 Training accuracy:	 0.96 Test accuracy:	 0.9646
Epoch:	 10 Training accuracy:	 1.0 Test accuracy:	 0.9662
Epoch:	 11 Training accuracy:	 1.0 Test accuracy:	 0.9665
Epoch:	 12 Training accuracy:	 0.98 Test accuracy:	 0.968
Epoch:	 13 Training accuracy:	 0.98 Test accuracy:	 0.9691
Epoch:	 14 Training accuracy:	 1.0 Test accuracy:	 0.9707
Epoch:	 15 Training accuracy:	 1.0 Test accuracy:	 0.9713
Epoch:	 16 Training accuracy:	 0.98 Test accuracy:	 0.9725
Epoch:	 17 Trai